In [1]:
import pandas as pd
import numpy as np
import implicit
import scipy.sparse as sp

c:\Users\benja\Documents\Universidad\2025-2\Recomendadores\Proyecto\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Metricas

In [2]:
df_train = pd.read_csv(
    "train", sep=",", names=["userid", "itemid", "rating"], header=None
)

df_train.rating = [1 if x >= 5 else 0 for x in df_train.rating]

df_train.head()

,userid,itemid,rating
0,14179,13601.0,1
1,37548,34300.0,1
2,796,2592.0,0
3,3041,949.0,1
4,2493,5114.0,1


In [3]:
df_test = pd.read_csv(
    "test", sep=",", names=["userid", "itemid", "rating"], header=None
)

df_test.head()

,userid,itemid,rating
0,506,37517.0,10
1,16392,7311.0,9
2,553,12471.0,5
3,13348,8937.0,6
4,276,35997.0,3


In [4]:
user_items = {}
itemset = set()

for row in df_train.itertuples():
    if row[1] not in user_items:
        user_items[row[1]] = []

    user_items[row[1]].append(row[2])
    itemset.add(row[2])

itemset = np.sort(list(itemset))

sparse_matrix = np.zeros((len(user_items), len(itemset)))

for i, items in enumerate(user_items.values()):
    sparse_matrix[i] = np.isin(itemset, items, assume_unique=True).astype(int)

matrix = sp.csr_matrix(sparse_matrix.T)

user_item_matrix = matrix.T.tocsr()

In [ ]:
user2row = {user_id: matrix_row for matrix_row, user_id in enumerate(user_items.keys())}

item2col = {item_id: matrix_col for matrix_col, item_id in enumerate(itemset)}
col2item = {matrix_col: item_id for item_id, matrix_col in item2col.items()}

In [6]:
model_als = implicit.als.AlternatingLeastSquares(factors=300)
model_als.fit(user_item_matrix, show_progress=False)

c:\Users\benja\Documents\Universidad\2025-2\Recomendadores\Proyecto\.venv\lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 12 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


In [ ]:
def get_recommendations(user_id, n):
    user_row = user2row[user_id]
    
    rec = model_als.recommend(user_row, user_item_matrix[user_row], n)[0]
    return np.array([col2item[col] for col in rec])

In [8]:
from evaluate import get_metrics

get_metrics(df_train, df_test, get_recommendations, k=10, alpha=0.2)

Evaluando usuarios: 100%|██████████| 18591/18591 [00:04<00:00, 4545.30it/s]

--- Métricas Globales de Evaluación ---
{
  "mean_recall": 0.023074160531513736,
  "mean_precision": 0.002504938049919196,
  "mean_ap (MAP)": 0.010869232172428436,
  "mean_ndcg": 0.013769247035573598,
  "mean_novelty": 8.49143249569358,
  "mean_diversity": 0.8387838587599917,
  "num_users_evaluated": 11138
}

--- Reporte de Fairness (Disparidad de Grupo) ---
{
  "delta_threshold": 0.2,
  "group_averages": {
    "Male": {
      "recall": 0.024538356673689428,
      "precision": 0.0026893047465609125,
      "MAP": 0.01142816816072486,
      "nDCG": 0.014547098198591426,
      "novelty": 8.50124437476057,
      "diversity": 0.8392127790499129,
      "count": 8069
    },
    "Female": {
      "recall": 0.019224503095470837,
      "precision": 0.0020202020202020206,
      "MAP": 0.009399680367422301,
      "nDCG": 0.011724124508890363,
      "novelty": 8.465635150567621,
      "diversity": 0.8376561436021629,
      "count": 3069
    }
  },
  "disparity_reports": [
    {
      "pair": [
    